In [1]:
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import os 

k_colors = ['rgb(0,127,206)','rgb(62,196,4)','rgb(255,125,16)','rgb(253,99,90)']#blue,gree,orange,red
plotly_themes = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]#plotly_white plotly_dark
theme = plotly_themes[2]


In [2]:
#path to data files
notebook_path = os.getcwd()
project_path = notebook_path[0:len(notebook_path)-16]+"data/"

In [3]:
genome_tags = pd.read_csv(project_path+'genome_tags.csv')
genome_tags.head(5)

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [4]:
genome_scores = pd.read_csv(project_path+'genome_scores.csv')
genome_scores .head(5)

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [5]:
movies = pd.read_csv(project_path+'movie.csv')
movies["year"] = movies["title"].str.extract('(\(\d\d\d\d\))',expand=False) #everything with parentheis and numbers (1995)
movies['year'] = movies["year"].str.extract('(\d\d\d\d)',expand=False) #only numbers
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')#replace numbers and parenthesis
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies.head(5)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


## Initial discovery of the genome tags

In [6]:
genomes = genome_scores.merge(genome_tags)

In [7]:
genomes.shape

(11709768, 4)

In [8]:
genome_scores.movieId.nunique()

10381

In [9]:
genome_scores.tagId.nunique()

1128

In [12]:
print("Min relevance: "+str(min(genome_scores.relevance))+ ",Max relevance: "+str(max(genome_scores.relevance)))

Min relevance: 0.0002499999999999,Max relevance: 1.0


In [13]:
toy_story_movie = genomes[genomes.movieId == 1]
toy_story_movie.groupby(["tag"]).agg(max_relevance=("relevance","max")).reset_index().sort_values('max_relevance',ascending =False).head()

,tag,max_relevance
1035,toys,0.99925
243,computer animation,0.99850
785,pixar animation,0.99600
588,kids and family,0.99075
63,animation,0.98575


There are 10,381 unique movies that were tag with 1128 unique tags with a range from 0 to 1. It seems that tags with similarity to certain movie have higher relevance.

It would be nice to know when this relevance were made. Because of leakage we will not include this information, but if we could know when this similarity matrix was made, it could be include to create comunity of movies with similar tags and have graph clusters having similar movies by relevant tags.